In [1]:
from torch import nn
import torch as t
from torch.nn import functional as F

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1, shortcut=None):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, 3, stride, 1, bias=False),
                nn.BatchNorm2d(outchannel),
                nn.ReLU(inplace=True),
                nn.Conv2d(outchannel, outchannel, 3, 1, 1, bias=False),
                nn.BatchNorm2d(outchannel))
        self.right = shortcut
    
    def forward(self, x):
        out = self.left(x)
        residual = x if self.right is None else self.right(x)
        out += residual
        return F.relu(out)
    

In [3]:
class ResNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet, self).__init__()
        self.pre = nn.Sequential(
                nn.Conv2d(3, 64, 7, 2, 3, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(3, 2, 1)) 
        self.layer1 = self._make_layer(64, 128, 3)
        self.layer2 = self._make_layer(128, 256, 4, stride=2)
        self.layer3 = self._make_layer(256, 512, 6, stride=2)
        self.layer4 = self._make_layer(512, 512, 3, stride=2)
        
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, inchannel, outchannel, block_num, stride=1):
        shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, 1, stride, bias=False),
                nn.BatchNorm2d(outchannel))
        layers = []
        layers.append(ResidualBlock(inchannel, outchannel, stride, shortcut))
        for i in range(1, block_num):
            layers.append(ResidualBlock(outchannel, outchannel))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.pre(x)
        
        x = self.layer1(x)
        
        x = self.layer2(x)
        
        x = self.layer3(x)
        
        x = self.layer4(x)
        
        x = F.avg_pool2d(x, 7)
        
        x = x.view(x.size(0), -1)
        
        return self.fc(x)
        
    

In [6]:
model = ResNet()
input = t.autograd.Variable(t.rand(1, 3, 224, 224))
o = model(input)

print(o)

tensor([[-1.8459e-01, -4.2741e-01,  2.7614e-01,  3.7189e-01, -4.5399e-02,
          1.0718e-01,  5.0221e-01,  1.2300e-01, -1.4207e-01,  5.1328e-02,
          2.6031e-01,  2.6959e-01,  1.3483e-01,  4.6587e-02, -3.0129e-01,
          9.2232e-02,  6.8730e-02,  2.2659e-01, -2.8990e-01, -1.9060e-02,
         -4.7979e-01,  2.7857e-01, -4.1864e-01, -6.2080e-01,  2.1216e-01,
         -6.1194e-01, -1.1612e-01, -3.7080e-01, -7.9602e-01, -4.2593e-01,
          1.1938e-01,  7.3531e-02,  9.4649e-02, -7.1536e-02, -2.0352e-02,
          2.8021e-01, -4.6204e-01,  2.8209e-01, -5.1016e-01, -3.2381e-01,
         -5.7069e-01,  2.7032e-01,  3.7346e-01,  5.2358e-01,  1.4972e-02,
         -4.7318e-01,  3.2507e-01,  4.3905e-01,  1.7684e-01,  4.9776e-02,
         -1.9836e-01, -5.1326e-01,  3.3318e-01,  2.6965e-02,  2.7188e-01,
         -1.8486e-01,  4.3728e-01, -7.1503e-01, -1.2959e-01,  7.0619e-02,
         -1.7100e-01,  2.2631e-01, -3.1946e-01, -3.1629e-01,  2.0002e-02,
         -4.5008e-01, -2.3605e-01,  3.

In [7]:
from torchvision import models
model = models.resnet34()